In [1]:
!pip install transformers datasets trl huggingface_hub bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.2 MB/s eta 0:00:00:00:0100:01


In [2]:
from huggingface_hub import login
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
WANDB_API_KEY = user_secrets.get_secret("WANDB_API_KEY")

login(token=HF_TOKEN)
wandb.login(key=WANDB_API_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dinhvuha2003 (dinhvuha2003-ho-chi-minh-city-university-of-technology). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch


# # Khởi tạo wandb
wandb.init(project="Qwen2.5-7B-Instruct-Medical-Chatbot", name="1-epoch")


model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Hỗ trợ nhiều GPU
    # quantization_config=bnb_config  # Áp dụng cấu hình 8-bit
    torch_dtype=torch.bfloat16,  # Reduce memory usage
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
finetune_name = "Medical"


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# Generate the train_prompt:

In [11]:
EOS_TOKEN = tokenizer.eos_token  # Ensure EOS token is included

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]  
    outputs = examples["Response"]
    texts = []

    for input, cot, output in zip(inputs,  cots, outputs):
        messages = [
            {"role": "system", "content": "You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. Please answer the following medical question."},
            {"role": "user", "content": f"Question: {input}"},
            {"role": "assistant", "content": f"Reasoning:\n{cot}\nFinal Answer:\n{output}"}
        ]

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False) 
        texts.append(text)

    return {"text": texts}

# Split data:

In [13]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:240]",trust_remote_code=True)
# split_dataset = dataset.train_test_split(test_size=0.2)

train_dataset = dataset
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
# val_dataset = val_dataset.map(formatting_prompts_func, batched=True)
# print("val data example:")
# print(val_dataset["text"][0])
print("train data example:")
print(train_dataset["text"][0])
print(train_dataset["text"][1])



Map:   0%|          | 0/480 [00:00<?, ? examples/s]

train data example:
<|im_start|>system
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. Please answer the following medical question.<|im_end|>
<|im_start|>user
Question: A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?<|im_end|>
<|im_start|>assistant
Reasoning:
Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability

# Setting up models


In [14]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 8
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 8
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

In [15]:
args = SFTConfig(
    # Output settings
    output_dir=finetune_name,  # Directory to save model checkpoints
    # Training duration
    num_train_epochs=1,  # Number of training epochs
    dataset_text_field="text",# Focus on what column we train
    # Batch size settings
    max_steps=60,
    per_device_train_batch_size=2,  # Batch size per GPU
    # per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch
    # Memory optimization
    gradient_checkpointing=True,  # Trade compute for memory savings
    # Optimizer settings
    optim="adamw_torch_fused",  # Use fused AdamW for efficiency
    learning_rate=2e-4,  # Learning rate (QLoRA paper)
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Learning rate schedule
    warmup_ratio=0.03,  # Portion of steps for warmup
    lr_scheduler_type="constant",  # Keep learning rate constant after warmup
    # Logging and saving
    logging_steps=10,  # Log metrics every N steps
    save_steps=10,  # Save checkpoint every epoch
    # save_strategy="epoch", 
    # eval_strategy="steps",
    
    # Precision settings
    bf16=True,  # Use bfloat16 precision
    # Integration settings
    push_to_hub=False,  # Don't push to HuggingFace Hub
    report_to="wandb",  # Disable external logging
)

In [16]:
from transformers import TrainingArguments, EarlyStoppingCallback
max_seq_length = 2160  # max sequence length for model and packing of the dataset
tokenizer.model_max_length = max_seq_length  # Adjust the sequence length
# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    peft_config=peft_config,  # LoRA configuration
    processing_class=tokenizer,
    # max_seq_length=max_seq_length,  # Maximum sequence length
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Dừng nếu không cải thiện sau 3 epoch
    # packing=True,  # Enable input packing for efficiency
    # dataset_kwargs={
    #     "add_special_tokens": False,  # Special tokens handled by template
    #     "append_concat_token": False,  # No additional separator needed
    # },
)

Converting train dataset to ChatML:   0%|          | 0/480 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2529 > 2160). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/480 [00:00<?, ? examples/s]

# Train model

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model("Medical/final")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
model_id_ESI = "vuha2003/Qwen2.5-7B-Instruct-Medical"
model = AutoModelForCausalLM.from_pretrained(
    model_id_ESI,
    device_map="auto",  # Enables multi-GPU support
    torch_dtype=torch.bfloat16,  # Reduce memory usage
)
tokenizer = AutoTokenizer.from_pretrained(model_id_ESI)

In [ ]:
def generate_response(question):
    """Formats the question and generates a response from the fine-tuned model."""
    # Create chat format input
    messages = [
        {"role": "system", "content": "You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.  Please answer the following medical question."},
        {"role": "user", "content": f"Question: {question}"}
    ]
    # Convert messages to model input using the chat template
    model_input = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    # Move input to the correct device
    model_input = model_input.to(model.device)
    # Generate response
    with torch.no_grad():
        output_ids = model.generate(model_input, max_new_tokens=2080, temperature=0.6, top_p=0.9, do_sample=True)
    # Decode the response
    response_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Extract Assistant's response
    response_parts = response_text.split("assistant\n")
    if len(response_parts) > 1:
        return response_parts[-1].strip()  # Return only the assistant's message
    return response_text.strip()

In [ ]:
# Example usage
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"
response = generate_response(question)
print(response)